In [1]:
from db_connection import DB_Connection
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.widgets import TextBox
import re, json
from wordcloud import WordCloud
import Tweet


db = DB_Connection()

database = db.query('''SELECT * FROM TWTTWEET WHERE TWEET_CREATED<'2021-01-01'
                    ORDER BY TWEET_CREATED DESC;''')
df = pd.DataFrame(database, columns=['id','userid','text','date',
                            'link','media1','media2','media3', 'media4'])
df = df.loc[:, ['id', 'text']]


processed = Tweet.clean_text(df, 'text')
classified = Tweet.classify_text(processed, 'clean')

# incidents = classified[['id','isIncident']].groupby('isIncident').size()
# accidents = classified[['id','isAccident']].groupby('isAccident').size()[1]
# dangers = classified[['id','isDanger']].groupby('isDanger').size()[1]
# obstacles = classified[['id','isObstacle']].groupby('isObstacle').size()[1]

Connection succesful.


In [ ]:
joined = ' '.join([t for t in classified.clean])
joined = joined.split()
new_joined = pd.Series(joined)

In [ ]:
len(new_joined.unique())

In [ ]:
x = [0,1,2]
data = [accidents,obstacles,dangers]
plt.figure(figsize=(10,6))
g = sns.barplot(x=x, y=data)
plt.title('% de incidentes por categoría', fontsize=16)
plt.xticks(ticks=x,
        labels=['accidentes','obstáculos','peligros'],
        fontsize=13)
plt.ylim(top=65000)

for i, v in enumerate(data):
    g.text(i, v+1000, v, color='black', ha="center", fontsize=12)
    # plt.annotate(v, (i+3, 10), color='black', fontsize=12)
plt.show()

## Text classification

In [2]:
from sklearn.svm import LinearSVC
import pickle
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
classified.head(1)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer1 = CountVectorizer(max_features=600, min_df=5, max_df=0.7)
X11 = vectorizer1.fit_transform(classified.clean).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
X2 = tfidf.fit_transform(X1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfi = TfidfVectorizer(sublinear_tf=True, max_features=500, min_df=5, max_df=0.7, encoding='utf-8', ngram_range=(1,2))
X3 = tfi.fit_transform(classified.clean).toarray()
# labels = classified.isIncident

In [4]:
y = [i for i in classified[['isAccident','isObstacle','isDanger']].values]

In [5]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X11, y, test_size=0.2, random_state=0)

### Boosting Model

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn import metrics
import xgboost as xgb
import time

In [ ]:
from xgboost import XGBClassifier

start = time.time()
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
end = time.time()
print('Duration:', str(end-start))

y_xgb = xgb.predict(X_test)

print(confusion_matrix(y_test,y_xgb))
print(classification_report(y_test,y_xgb))
print(accuracy_score(y_test, y_xgb))

In [ ]:
xgresume = []

for i in [64,128,256,512,1024,2048,4096]:
    print('\nestimators:', str(i))
    start = time.time()
    xgb = XGBClassifier(n_estimators=i, learning_rate=0.3, eval_metric='error') # learning_rate
    xgb.fit(X_train, y_train)
    end = time.time()
    y_xgb = xgb.predict(X_test)
    print(str(end-start), accuracy_score(y_test, y_xgb))
    xgresume.append([i, end-start, accuracy_score(y_test, y_xgb),
    confusion_matrix(y_test,y_xgb), classification_report(y_test,y_xgb)])

In [ ]:
xs = [i[0] for i in xgresume]
ys = [i[2] for i in xgresume]
zs = [i[3] for i in xgresume]

plt.figure(figsize=(11,4))
plt.title('XGBoost')
plt.subplot(1, 2, 1)
sns.pointplot(x=xs, y=ys)
# sns.lineplot(x=range(len(xs)), y=ys, markers=True, style='event')
plt.title('Duración de entrenamiento')
plt.xticks(range(len(xs)), xs)
plt.subplot(1, 2, 2)
sns.pointplot(x=xs, y=zs)
plt.title('Precisión obtenida')
plt.xticks(range(len(xs)), xs)
plt.show()

In [8]:
param_test0 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

start = time.time()
gsearch0 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.3, n_estimators=100, max_depth=6,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, 
                                        eval_metric='error', seed=27), 
                        param_grid = param_test0, scoring='accuracy',n_jobs=4, cv=3, verbose=2)
grid_result0 = MultiOutputClassifier(gsearch0).fit(X_train1, y_train1)
end = time.time()
print(str(end-start))
with open('param_test0', 'wb') as picklefile:
    pickle.dump(grid_result0, picklefile)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
6813.413553237915


In [31]:
for i in range(3):
    grid_results0 = pd.DataFrame(grid_result0.estimators_[i].cv_results_)
    grid_results0.to_csv('grid_results0-%d.csv'%(i))

In [33]:
grid_result0.estimators_[2].cv_results_

{'max_depth': 3, 'min_child_weight': 1}

In [ ]:
param_test1 = {
 'gamma':[i/10.0 for i in range(0,4)]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.3, n_estimators=100,
                                        max_depth=gsearch0.best_params_.['max_depth'],
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test1, scoring='accuracy',n_jobs=4, cv=3, verbose=3)
gsearch1.fit(X_train1, y_train1)
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'subsample':[i/10.0 for i in range(6,11)],
 'colsample_bytree':[i/10.0 for i in range(6,11)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.3, n_estimators=100, max_depth=9,
                                        min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test2, scoring='accuracy',n_jobs=4, cv=3, verbose=3)
gsearch2.fit(X_train1, y_train1)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(8,11)],
 'colsample_bytree':[i/10.0 for i in range(8,11)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.3, n_estimators=100, max_depth=9,
                                        min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test4, scoring='accuracy',n_jobs=4, cv=3, verbose=1)
gsearch4.fit(X_train1, y_train1)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test3 = {
 'reg_alpha':[0, 1e-5, 0.001, 0.005, 0.01, 0.05, 0.1]
}

gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.3, n_estimators=100, max_depth=9,
                                        min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test3, scoring='accuracy',n_jobs=4, cv=3, verbose=3)
gsearch3.fit(X_train1, y_train1)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
grid_results3 = pd.DataFrame(gsearch3.cv_results_)
grid_results3.sort_values('rank_test_score').head(5)#[['rank_test_score','param_colsample_bytree', 'param_subsample', 'mean_test_score', 'std_test_score']].rename(columns={'param_colsample_bytree':'colsample_bytree', 'param_param_subsample':'subsample'})

In [ ]:
grid_results3.to_csv('grid_search_results3-4.csv')

In [ ]:
xgb1 = XGBClassifier(
    learning_rate=0.3,
    n_estimators=2000,
    max_depth=9, min_child_weight=1,
    gamma=0.1, subsample=0.9, colsample_bytree=1,
    nthread=4, seed=27, verbosity=3)

xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb1.get_params()['n_estimators'], nfold=5,
    early_stopping_rounds=50)
xgb1.set_params(n_estimators=cvresult.shape[0])

In [ ]:
#Fit the algorithm on the data
print('Start training.')
start = time.time()
xgb1.fit(X_train, y_train, verbose=True)
print('End training.') 
end = time.time()
    
#Predict training set:
dtrain_predictions = xgb1.predict(X_test)
dtrain_predprob = xgb1.predict_proba(X_test)[:,1]
    
#Print model report:
print("\nModel Report")
print("Duration:", str(end-start))
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, dtrain_predictions))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, dtrain_predprob))
                
plt.figure(figsize=(15,5))
feat_imp = pd.Series(xgb1.get_booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

#### Estimators adjust

In [ ]:
xgresume = []
xgb_arr = []

for i in [512,1024,2048]:
    print('\nestimators:', str(i))
    start = time.time()
    xgb = XGBClassifier(
        learning_rate=0.3,
        n_estimators=i,
        max_depth=9, min_child_weight=1,
        gamma=0.1, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.005,
        nthread=4, seed=27, eval_metric='error', verbosity=0)
    xgb_arr.append(xgb)
    print('Fitting... ', time.asctime())
    xgb_arr[-1].fit(X_train1, y_train1)
    end = time.time()
    print('Predicting... ', time.asctime())
    y_xgb = xgb_arr[-1].predict(X_test1)
    print(str(end-start), accuracy_score(y_test1, y_xgb))
    xgresume.append([i, end-start, accuracy_score(y_test1, y_xgb),
    confusion_matrix(y_test1,y_xgb), classification_report(y_test1,y_xgb)])

In [ ]:
for i in xgresume:
    for s in i: print(s)
    print('\n')

In [ ]:
xs = [i[0] for i in xgresume]
ys = [i[1] for i in xgresume]
zs = [i[2] for i in xgresume]

plt.figure(figsize=(9,3.5))
plt.suptitle('Resultados del ajuste de número de estimados', fontsize=16)
plt.subplot(1, 2, 1)
sns.pointplot(x=xs, y=ys)
plt.title('Duración de entrenamiento')
plt.xticks(range(len(xs)), xs)
plt.subplot(1, 2, 2)
sns.pointplot(x=xs, y=zs)
plt.title('Precisión obtenida')
plt.xticks(range(len(xs)), xs)
plt.tight_layout()
plt.show()

In [ ]:
with open('xgbresume0', 'wb') as picklefile:
    pickle.dump(xgresume, picklefile)
with open('xgbarr0', 'wb') as picklefile:
    pickle.dump(xgb_arr, picklefile)

In [ ]:
xgresume1 = []
xgb_arr1 = []

for i in [600,700,800,900]:
    print('\nestimators:', str(i))
    start = time.time()
    xgb = XGBClassifier(
        learning_rate=0.3,
        n_estimators=i,
        max_depth=9, min_child_weight=1,
        gamma=0.1, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.005,
        nthread=4, seed=27, eval_metric='error', verbosity=0)
    xgb_arr1.append(xgb)
    xgb_arr1[-1].fit(X_train1, y_train1)
    end = time.time()
    y_xgb = xgb_arr1[-1].predict(X_test1)
    print(str(end-start), accuracy_score(y_test1, y_xgb))
    xgresume1.append([i, end-start, accuracy_score(y_test1, y_xgb),
    confusion_matrix(y_test1,y_xgb), classification_report(y_test1,y_xgb)])

In [ ]:
for i in xgresume1:
    for s in i: print(s)
    print('\n')

In [ ]:
[i[0] for i in xgresume1] + [xgresume[1][0]]

In [ ]:
xs = [i[0] for i in xgresume1] + [xgresume[1][0]]
ys = [i[1] for i in xgresume1] + [xgresume[1][1]]
zs = [i[2] for i in xgresume1] + [xgresume[1][2]]

plt.figure(figsize=(8.5,3.5))
plt.suptitle('Resultados del subajuste de número de estimados', fontsize=16)
plt.subplot(1, 2, 1)
sns.pointplot(x=xs, y=ys)
plt.title('Duración de entrenamiento')
plt.xticks(range(len(xs)), xs)
plt.subplot(1, 2, 2)
sns.pointplot(x=xs, y=zs)
plt.title('Precisión obtenida')
plt.xticks(range(len(xs)), xs)
plt.tight_layout()
plt.show()

In [ ]:
with open('xgbresume1', 'wb') as picklefile:
    pickle.dump(xgresume1, picklefile)
with open('xgbarr1', 'wb') as picklefile:
    pickle.dump(xgb_arr1, picklefile)

In [ ]:
print('\nestimators: 1000')
start = time.time()
xgb_final = XGBClassifier(
    learning_rate=0.3,
    n_estimators=1000,
    max_depth=9, min_child_weight=1,
    gamma=0.1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005,
    nthread=4, seed=27, eval_metric='error', verbosity=0)
xgb_final.fit(X_train, y_train)
end = time.time()
y_xgb = xgb_final.predict(X_test)
print(str(end-start), accuracy_score(y_test, y_xgb))

In [ ]:
xgb_final = xgb_arr1[-1]
y_xgb = xgb_final.predict(X_test1)
xgb_final

In [ ]:
xgb_con = confusion_matrix(y_test1,y_xgb)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                xgb_con.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     xgb_con.flatten()/np.sum(xgb_con)]
labels = ['%s\n%s\n%s' % (v1, v2, v3) for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(6,3.2))
sns.heatmap(xgb_con/np.sum(xgb_con), annot=labels, fmt='', cmap='Blues')
plt.title('Matriz de confusión para 900 estimados', fontsize=13)
plt.show()

In [ ]:
with open('xgb_classifier', 'wb') as picklefile:
    pickle.dump(xgb_final, picklefile)

#### Count vectors adjust

In [ ]:
# load model
with open('xgb_classifier', 'rb') as training_model:
    xgb = pickle.load(training_model)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

xgresume2 = []
xgb_arr2 = []

for i in [800, 1000, 2000, 5000, 10000]:
    print('\nnumber of features:', str(i))
    vectorizer = CountVectorizer(max_features=i, min_df=5, max_df=0.7)
    print('Fitting vectorizer...')
    X1 = vectorizer.fit_transform(classified.clean).toarray()
    print('Creating train-test data...')
    X_train, X_test, y_train, y_test = train_test_split(X1, classified.isIncident, test_size=0.2, random_state=0)
    print('Fitting model...', time.asctime())
    with open('xgb_classifier', 'rb') as training_model:
        xgb_arr2.append(pickle.load(training_model))
    start = time.time()
    xgb_arr2[-1].fit(X_train,y_train)
    end = time.time()
    print('Fitting time:', str(end-start), '\nPredicting...')
    y_xgb = xgb_arr2[-1].predict(X_test)
    print(accuracy_score(y_test, y_xgb))
    print(confusion_matrix(y_test,y_xgb))
    print(classification_report(y_test,y_xgb))
    xgresume2.append([i, end-start, accuracy_score(y_test, y_xgb),
    confusion_matrix(y_test,y_xgb), classification_report(y_test,y_xgb)])

In [ ]:
with open('xgbresume2', 'wb') as picklefile:
    pickle.dump(xgresume2, picklefile)
with open('xgbarr2', 'wb') as picklefile:
    pickle.dump(xgb_arr2, picklefile)

In [ ]:
# load model
with open('./others/resumes/xgbarr2', 'rb') as arr:
    xgb_arr2 = pickle.load(arr)
with open('./others/resumes/xgbresume2', 'rb') as resume:
    xgresume2 = pickle.load(resume)

In [ ]:
xs = [i[0] for i in xgresume2]
ys = [i[1] for i in xgresume2]
zs = [i[2] for i in xgresume2]

plt.figure(figsize=(9,3.5))
plt.suptitle('Resultados del ajuste de número de vectores', fontsize=16)
plt.subplot(1, 2, 1)
sns.pointplot(x=xs, y=ys)
plt.title('Duración de entrenamiento')
plt.xticks(range(len(xs)), xs)
plt.subplot(1, 2, 2)
sns.pointplot(x=xs, y=zs)
plt.title('Precisión obtenida')
plt.xticks(range(len(xs)), xs)
plt.tight_layout()
plt.show()

In [ ]:
# save final xgb model
xgb_final2 = xgb_arr2[-1]

In [ ]:
# fit vectorizer 5000 features
i = 5000
print('\nnumber of features:', str(i))
vectorizer10 = CountVectorizer(max_features=i, min_df=5, max_df=0.7)
print('Fitting vectorizer...', time.asctime())
vectorizer10.fit(classified.clean)

In [ ]:
# convert and split data
X_vect = vect.transform(classified.clean).toarray()
print('Creating train-test data...')
X_train, X_test, y_train, y_test = train_test_split(X_vect, classified.isIncident, test_size=0.2, random_state=0)
print('Done.')

In [ ]:
with open('vectorizer5000', 'wb') as picklefile:
    pickle.dump(vectorizer10, picklefile)

In [ ]:
# load model fo 5000 features
with open('count_vectorizer', 'rb') as v:
    vect = pickle.load(v)

In [ ]:
#load model
with open('xgb_5000', 'rb') as training_model:
    xgb_classifier = pickle.load(training_model)

In [ ]:
# fit classifier with 5000 features
print('Fitting model...', time.asctime())
start = time.time()
xgb_classifier.fit(X_train,y_train)
end = time.time()
print('Fitting time:', str(end-start), '\nPredicting...')

In [ ]:
# predict test data
print('Predicting data...')
y_xgb2 = xgb_classifier.predict(X_test)
print(accuracy_score(y_test, y_xgb2))
print(confusion_matrix(y_test,y_xgb2))
print(classification_report(y_test,y_xgb2))
# xgresume2.append([i, end-start, accuracy_score(y_test, y_xgb2),
# confusion_matrix(y_test,y_xgb2), classification_report(y_test,y_xgb2)])

In [ ]:
# save trained classifier
with open('xgb_features_final', 'wb') as picklefile:
    pickle.dump(xgb_classifier, picklefile)

In [ ]:
# count vectors for 10000
i = 10000
print('\nnumber of features:', str(i))
vectorizer100 = CountVectorizer(max_features=i, min_df=5, max_df=0.7)
print('Fitting vectorizer...', time.asctime())
vectorizer100.fit(classified.clean)
X_vect1 = vectorizer100.transform(classified.clean).toarray()
print('Creating train-test data...')
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_vect1, classified.isIncident, test_size=0.2, random_state=0)
with open('vectorizer10000', 'wb') as picklefile:
    pickle.dump(vectorizer100, picklefile)

In [ ]:
# fitting 10000 vectors
#load model
with open('xgb_classifier', 'rb') as model:
    xgb_classifier1 = pickle.load(model)
print('Fitting model...', time.asctime())
start = time.time()
xgb_classifier1.fit(X_train3,y_train3)
end = time.time()
print('Fitting time:', str(end-start), '\nPredicting...')
y_xgb3 = xgb_classifier1.predict(X_test3)
print(accuracy_score(y_test3, y_xgb3))
print(confusion_matrix(y_test3,y_xgb3))
print(classification_report(y_test3,y_xgb3))
# xgresume2.append([i, end-start, accuracy_score(y_test, y_xgb3),
# confusion_matrix(y_test,y_xgb3), classification_report(y_test,y_xgb3)])

In [ ]:
xgb_con = confusion_matrix(y_test,y_xgb2)
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                xgb_con.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     xgb_con.flatten()/np.sum(xgb_con)]
labels = ['%s\n%s\n%s' % (v1, v2, v3) for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(6,3.2))
sns.heatmap(xgb_con/np.sum(xgb_con), annot=labels, fmt='', cmap='Blues')
plt.title('Matriz de confusión para 5000 características', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
feat_df = pd.DataFrame(X_vect)
feat_df.columns = vect.get_feature_names()

plt.figure(figsize=(10,3.5))
feat_imp = pd.Series(xgb_classifier.get_booster().get_fscore()).sort_values(ascending=False)[:40]
# feat_imp.plot(kind='bar', title='Feature] Importances')
feat_values = [int(re.sub('f', '', i)) for i in feat_imp.index]
feat_names = [feat_df.columns[i] for i in feat_values]
features = pd.DataFrame({'feature_num':feat_values, 'names':feat_names, 'count':feat_imp.values})
# plt.bar(x=feat_values, height=feat_imp.values)
# sns.barplot(x=feat_values, y=feat_imp.values, data=feat_imp, order=feat_imp.values)
plt.suptitle('Importancia de características', fontsize=16)
# plt.subplot(2, 1, 1)
sns.barplot(x='names', y='count', data=features)
plt.xticks(rotation=70, fontsize=12, ha='right')
plt.ylabel('Puntaje de características', fontsize=12)
# plt.tight_layout()
plt.xlabel(None)